# Class definition

In [1]:
import os
from pypdf import PdfReader
import pandas as pd
import re
from abc import ABC, abstractmethod

In [43]:
class PageIdentifier(ABC):
    def __init__(self, base_dir, result_dir, verbose=False):
        self.base_dir = base_dir
        self.result_dir = result_dir
        self.verbose = verbose
        self.pages = {
            "Aktiva": [],
            "Passiva": [],
            "GuV": []
        }
        
    def walk_dir(self):
        for root, _, files in os.walk(self.base_dir):
            for file in files:
                if file.endswith(".pdf"):
                    filepath = os.path.join(root, file)
                    if self.verbose:
                        print(filepath)
                    
                    try:
                        self.identify_pages(filepath)
                    except Exception as e:
                        print(f"Error reading {filepath}: {e}")

    def __generate_dataframe(self):
        df_list = []
        for item in self.pages.keys():
            df = pd.DataFrame.from_records(self.pages[item], columns=['filepath', 'page'])
            df['type'] = item
            df_list.append(df)

        df = pd.concat(df_list, ignore_index=True)
        return(df)
    
    def __generate_benchmarking_dataframe(self):
        df = self.__generate_dataframe()

        df_truth = pd.read_csv("../Python/table_page_truth.csv")
        df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type').reset_index(drop=True)
        
        df_temp = df_truth.join(
            df.set_index(['filepath', 'type']),
            on=['filepath', 'type'],
            how='left',
            rsuffix='_identified'
        )
        df_temp['company'] = df_temp['filepath'].str.split('/').str[2]
        df_temp['file'] = df_temp['filepath'].str.split('/').str[3]
        df_temp = df_temp.drop(columns=['filepath'])

        return df_temp

    def save_to_csv(self):
        df = self.__generate_dataframe()
        df.to_csv(self.result_dir + "_identified_pages.csv", index=False)
        
    def get_dataframe(self):
        df = self.__generate_dataframe()
        return df

    def benchmark(self, verbose=True):
        df_temp = self.__generate_benchmarking_dataframe()

        correct_identified = df_temp[df_temp['page'] == df_temp['page_identified']]
        wrong_identified = df_temp[(df_temp['page'] != df_temp['page_identified']) & (df_temp['page_identified'].notna())]
        missing = df_temp[df_temp['page_identified'].isna()]

        if verbose:
            print(f"There are {df_temp.drop_duplicates(subset=['file', 'type', 'company', 'page']).shape[0]} unique tables.")
            print(f"Correctly identified: {len(correct_identified)}")
            print(f"Wrongly identified: {len(wrong_identified)}")
            print(f"Missing: {len(missing)}")

        return {'correct': correct_identified, 'wrong': wrong_identified, 'missing': missing}
        
    
    @abstractmethod
    def identify_pages(self, filepath):
        pass

In [44]:
from pytesseract import image_to_string
from pdf2image import convert_from_path

class PageIdentifierRegex(PageIdentifier):
    def __init__(self, base_dir, result_dir, regex_patterns, verbose=False, ocr=False):
        super().__init__(base_dir, result_dir, verbose)
        self.regex_patterns = regex_patterns
        self.ocr = ocr

    def identify_pages(self, filepath):
        reader = PdfReader(filepath)
        for page_num, page in enumerate(reader.pages):
            text = page.extract_text()

            if not text.strip() and self.ocr:
                # If no text is found and OCR is enabled, perform OCR
                if self.verbose:
                    print(f"Performing OCR on {filepath} page {page_num + 1}")
                images = convert_from_path(filepath, first_page=page_num + 1, last_page=page_num + 1)
                if images:
                    text = image_to_string(images[0])

            for item in self.pages.keys():
                if all(
                    re.search(term, text, re.IGNORECASE | re.S) if isinstance(term, str) else False
                    for term in self.regex_patterns[item]
                ):
                    self.pages[item].append((filepath, page_num + 1))
                    if self.verbose:
                        print(f"Found {item} in {filepath} on page {page_num + 1}")


# Benchmarking

## Simple Rexeg OCR

In [ ]:
simple_regex_patterns = {
    "Aktiva": [
        r"aktiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "Passiva": [
        r"passiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))"
    ]
}

page_identifier_regex_dummy = PageIdentifierRegex(
    base_dir = "../Geschaeftsberichte/degewo AG/",
    # base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_1",
    regex_patterns = simple_regex_patterns,
    verbose = False,
    ocr = True
)

# page_identifier_regex_dummy.walk_dir()

Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 1
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 2
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 3
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 4
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 5
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 6
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 7
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 8
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 9
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 10
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 11
Performing OCR on .

In [ ]:
ben_df_dummy = page_identifier_regex_dummy.benchmark()
ben_df_dummy['missing'][ben_df_dummy['missing']['company'] == 'degewo AG']

Correctly identified: 12
Wrongly identified: 48
Missing: 249


,page,type,page_identified,company,file
202,28,Aktiva,NaN,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
203,29,Passiva,NaN,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...


In [188]:
images = convert_from_path("../Geschaeftsberichte/degewo AG/201231_degewo_Konzernlagebericht_Konzernabschluss.pdf", first_page=27 + 1, last_page=27 + 1)
if images:
    text = image_to_string(images[0])

text
re.search(r"((20\d{2}).*(20\d{2}))", text, re.IGNORECASE | re.S)

<re.Match object; span=(65, 1938), match='2020\n\nAKTIVA\n\nA. Anlagevermégen\nI. Immaterie>

## Simple Regex

In [45]:
simple_regex_patterns = {
    "Aktiva": [
        r"aktiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "Passiva": [
        r"passiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))"
    ]
}

page_identifier_regex_1 = PageIdentifierRegex(
    base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    # base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_1",
    regex_patterns = simple_regex_patterns,
    verbose = False
)

page_identifier_regex_1.walk_dir()

In [46]:
ben_df1 = page_identifier_regex_1.benchmark()
ben_df1['missing']

There are 264 unique tables.
Correctly identified: 8
Wrongly identified: 52
Missing: 249


,page,type,page_identified,company,file
0,10,Aktiva,NaN,Berliner Bäder Betriebe,GB_BBB_Infra_2022_low.pdf
1,11,Passiva,NaN,Berliner Bäder Betriebe,GB_BBB_Infra_2022_low.pdf
2,13,GuV,NaN,Berliner Bäder Betriebe,GB_BBB_Infra_2022_low.pdf
3,28,Aktiva,NaN,Berliner Bäder Betriebe,GB_BBB_Infra_2022_low.pdf
4,29,Passiva,NaN,Berliner Bäder Betriebe,GB_BBB_Infra_2022_low.pdf
...,...,...,...,...,...
259,44,GuV,NaN,GESOBAU AG,GESOBAU_Geschaeftsbericht_2016.pdf
260,62,GuV,NaN,GESOBAU AG,GESOBAU_Geschaeftsbericht_2016.pdf
261,52,Aktiva,NaN,GESOBAU AG,GESOBAU_Geschaeftsbericht_2017.pdf
262,53,Passiva,NaN,GESOBAU AG,GESOBAU_Geschaeftsbericht_2017.pdf


In [47]:
from transformers import pipeline
from PIL import ImageDraw
# import warnings

threshold = 0.95

# warnings.filterwarnings("ignore")
pipe = pipeline("object-detection", model="microsoft/table-transformer-detection")

counter = {'with_table': 0, 'without_table': 0}

for _, row in ben_df1['correct'].iterrows():
    filepath = f"../Geschaeftsberichte/{row['company']}/{row['file']}"
    page_num = row['page']

    images = convert_from_path(filepath, first_page=page_num, last_page=page_num)
    results = pipe(images[0], threshold=threshold)
    
    # Draw the bounding boxes on the image
    # draw = ImageDraw.Draw(images[0])
    # for result in results:
    #     box = result['box']
    #     draw.rectangle([box['xmin'], box['ymin'], box['xmax'], box['ymax']], outline="red", width=3)

    #     # Show the image with the bounding boxes
    #     images[0].show()

    if len(results) > 0:
        counter['with_table'] += 1
    else:
        counter['without_table'] += 1

print(counter)

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'with_table': 8, 'without_table': 0}


In [51]:
df_truth = pd.read_csv("../Python/table_page_truth.csv")
df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type').reset_index(drop=True)

In [57]:
df = page_identifier_regex_1.get_dataframe()
df

,filepath,page,type
0,../Geschaeftsberichte/Berlin Energie und Netzh...,29,Aktiva
1,../Geschaeftsberichte/Berlin Energie und Netzh...,12,Aktiva
2,../Geschaeftsberichte/Berlin Energie und Netzh...,42,Aktiva
3,../Geschaeftsberichte/Berlin Energie und Netzh...,11,Aktiva
4,../Geschaeftsberichte/Berlin Energie und Netzh...,42,Aktiva
5,../Geschaeftsberichte/Berlin Energie und Netzh...,29,Passiva
6,../Geschaeftsberichte/Berlin Energie und Netzh...,12,Passiva
7,../Geschaeftsberichte/Berlin Energie und Netzh...,42,Passiva
8,../Geschaeftsberichte/Berlin Energie und Netzh...,11,Passiva
9,../Geschaeftsberichte/Berlin Energie und Netzh...,42,Passiva


In [58]:
df_common = df_truth.merge(df, on=['filepath', 'page', 'type'], how='inner')
df = df.merge(df_common[['filepath', 'page', 'type']], on=['filepath', 'page', 'type'], how='left', indicator=True)
df = df[df['_merge'] == 'left_only'].drop(columns=['_merge'])
df


,filepath,page,type
0,../Geschaeftsberichte/Berlin Energie und Netzh...,29,Aktiva
1,../Geschaeftsberichte/Berlin Energie und Netzh...,12,Aktiva
3,../Geschaeftsberichte/Berlin Energie und Netzh...,11,Aktiva
5,../Geschaeftsberichte/Berlin Energie und Netzh...,29,Passiva
6,../Geschaeftsberichte/Berlin Energie und Netzh...,12,Passiva
8,../Geschaeftsberichte/Berlin Energie und Netzh...,11,Passiva
10,../Geschaeftsberichte/Berlin Energie und Netzh...,3,GuV
12,../Geschaeftsberichte/Berlin Energie und Netzh...,13,GuV
13,../Geschaeftsberichte/Berlin Energie und Netzh...,16,GuV
14,../Geschaeftsberichte/Berlin Energie und Netzh...,3,GuV


In [59]:
df_common['page_identified'] = df_common['page']
df_common

,filepath,page,type,page_identified
0,../Geschaeftsberichte/Berlin Energie und Netzh...,10,GuV,10
1,../Geschaeftsberichte/Berlin Energie und Netzh...,23,GuV,23
2,../Geschaeftsberichte/Berlin Energie und Netzh...,42,Aktiva,42
3,../Geschaeftsberichte/Berlin Energie und Netzh...,42,Passiva,42
4,../Geschaeftsberichte/Berlin Energie und Netzh...,43,GuV,43
5,../Geschaeftsberichte/Berlin Energie und Netzh...,23,GuV,23
6,../Geschaeftsberichte/Berlin Energie und Netzh...,42,Aktiva,42
7,../Geschaeftsberichte/Berlin Energie und Netzh...,42,Passiva,42


In [ ]:
from transformers import pipeline
from PIL import ImageDraw, ImageFont
# import warnings

font = ImageFont.truetype(font="/usr/share/fonts/truetype/ubuntu/Ubuntu-B.ttf", size = 20)

# warnings.filterwarnings("ignore")
pipe = pipeline("object-detection", model="microsoft/table-transformer-detection")

counter = {'with_table': 0, 'without_table': 0}

for _, row in ben_df1['wrong'].iterrows():
    filepath = f"../Geschaeftsberichte/{row['company']}/{row['file']}"
    page_num = int(row['page_identified'])

    images = convert_from_path(filepath, first_page=page_num, last_page=page_num)
    results = pipe(images[0], threshold=threshold)
    
    if len(results) > 0:
        # Draw the bounding boxes on the image
        print(ben_df1['wrong'].loc[_])
        print(f"Found {len(results)} tables in {filepath} on page {page_num}")
        counter['with_table'] += 1
        draw = ImageDraw.Draw(images[0])
        for result in results:
            box = result['box']
            draw.rectangle([box['xmin'], box['ymin'], box['xmax'], box['ymax']], outline="red", width=3)
            draw.text((box['xmin']+10, box['ymin']+10), f"{filepath} at page: {page_num} ({result['score']:.2f})", fill="red", font=font)

            # Show the image with the bounding boxes
            images[0].show()
    else:
        counter['without_table'] += 1

print(counter)    

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


                                               209
page                                             9
type                                        Aktiva
page_identified                               29.0
company             Berlin Energie und Netzholding
file             Jahresabschluss_2021_BEN_GmbH.pdf
Found 1 tables in ../Geschaeftsberichte/Berlin Energie und Netzholding/Jahresabschluss_2021_BEN_GmbH.pdf on page 29
                                               210
page                                             9
type                                       Passiva
page_identified                               29.0
company             Berlin Energie und Netzholding
file             Jahresabschluss_2021_BEN_GmbH.pdf
Found 1 tables in ../Geschaeftsberichte/Berlin Energie und Netzholding/Jahresabschluss_2021_BEN_GmbH.pdf on page 29
Opening in existing browser session.
     page type  page_identified                         company  \
211    10  GuV              3.0  Berlin Energie und 

[59366:59366:0509/173119.974646:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 1 times!
[59366:59366:0509/173119.987412:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 2 times!


     page type  page_identified                         company  \
214    23  GuV              3.0  Berlin Energie und Netzholding   
214    23  GuV             21.0  Berlin Energie und Netzholding   
214    23  GuV             27.0  Berlin Energie und Netzholding   
214    23  GuV             28.0  Berlin Energie und Netzholding   
214    23  GuV             30.0  Berlin Energie und Netzholding   
214    23  GuV             42.0  Berlin Energie und Netzholding   
214    23  GuV             43.0  Berlin Energie und Netzholding   
214    23  GuV             47.0  Berlin Energie und Netzholding   
214    23  GuV             51.0  Berlin Energie und Netzholding   

                                  file  
214  Jahresabschluss-BEN-GmbH-2023.pdf  
214  Jahresabschluss-BEN-GmbH-2023.pdf  
214  Jahresabschluss-BEN-GmbH-2023.pdf  
214  Jahresabschluss-BEN-GmbH-2023.pdf  
214  Jahresabschluss-BEN-GmbH-2023.pdf  
214  Jahresabschluss-BEN-GmbH-2023.pdf  
214  Jahresabschluss-BEN-GmbH-2023.pdf  
2

[59366:59366:0509/173128.628582:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 3 times!


     page type  page_identified                         company  \
220    23  GuV              3.0  Berlin Energie und Netzholding   
220    23  GuV             27.0  Berlin Energie und Netzholding   
220    23  GuV             28.0  Berlin Energie und Netzholding   
220    23  GuV             29.0  Berlin Energie und Netzholding   
220    23  GuV             30.0  Berlin Energie und Netzholding   
220    23  GuV             42.0  Berlin Energie und Netzholding   
220    23  GuV             47.0  Berlin Energie und Netzholding   
220    23  GuV             51.0  Berlin Energie und Netzholding   

                                                  file  
220  SIGNATURE_DEE00084236.1.1_550912_Endexemplar_2...  
220  SIGNATURE_DEE00084236.1.1_550912_Endexemplar_2...  
220  SIGNATURE_DEE00084236.1.1_550912_Endexemplar_2...  
220  SIGNATURE_DEE00084236.1.1_550912_Endexemplar_2...  
220  SIGNATURE_DEE00084236.1.1_550912_Endexemplar_2...  
220  SIGNATURE_DEE00084236.1.1_550912_Endexemplar_2...

Created TensorFlow Lite XNNPACK delegate for CPU.
Attempting to use a delegate that only supports static-sized tensors with a graph that has dynamic-sized tensors (tensor#-1 is a dynamic-sized tensor).


## Expended Regex

In [171]:
regex_patterns_2 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_2 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_2",
    regex_patterns = regex_patterns_2,
    verbose = False
)

page_identifier_regex_2.walk_dir()

In [ ]:
ben_df2 = page_identifier_regex_2.benchmark()
ben_df2['missing']

Correctly identified: 235
Wrongly identified: 805
Missing: 12


,page,type,page_identified,company,file
178,26,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
179,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
185,17,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2007_BBB.pdf
187,24,Aktiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
188,25,Passiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
189,27,GuV,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
190,26,Aktiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
191,27,Passiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
192,29,GuV,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
193,26,Aktiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...


## Exhaustive Regex

In [308]:
# Aktiva und Passiva in einigen AfStatBBB nicht korrekt formatiert
regex_patterns_3 = {
    "Aktiva": [
        r"a\s*k\s*t\s*i\s*v\s*a|a\s*k\s*t\s*i\s*v\s*s\s*e\s*i\s*t\s*e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"p\s*a\s*s\s*s\s*i\s*v\s*a|p\s*a\s*s\s*s\s*i\s*v\s*s\s*e\s*i\s*t\s*e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn|guv",
        r"verlust|guv",
        r"rechnung|guv",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_3 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_3,
    verbose = False
)

page_identifier_regex_3.walk_dir()

In [309]:
ben_df3 = page_identifier_regex_3.benchmark()
ben_df3['missing']

There are 264 unique tables.
Correctly identified: 246
Wrongly identified: 1666
Missing: 9


,page,type,page_identified,company,file
186,24,Aktiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
187,25,Passiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
188,27,GuV,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
189,26,Aktiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
190,27,Passiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
191,29,GuV,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
192,26,Aktiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
193,27,Passiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
194,29,GuV,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...


Findet nun alles, was kein OCR braucht, aber dafür auch doppelt so viel irrelevantes.

In [221]:
ben_df3['wrong'].groupby(['type', 'company']).agg(
    count=('page', 'count')
).sort_values('count', ascending=False)

count
type    company                                    
Passiva Berlinovo                               225
Aktiva  GESOBAU AG                              172
Passiva IBB                                     148
Aktiva  IBB                                     139
        Berlinovo                               129
        Berliner Bäder Betriebe                 117
GuV     Berlinovo                               105
Passiva GESOBAU AG                              105
GuV     Berliner Bäder Betriebe                  94
Passiva Berliner Bäder Betriebe                  83
GuV     IBB                                      80
        GESOBAU AG                               65
        Berlin Energie und Netzholding           40
        Amt für Statistik Berlin-Brandenburg     35
Aktiva  Berlin Energie und Netzholding           31
Passiva Berlin Energie und Netzholding           31
Aktiva  Amt für Statistik Berlin-Brandenburg     13
Passiva Amt für Statistik Berlin-Brandenburg      8
GuV     degewo AG                                 7
Passiva degewo AG                                 7
Aktiva  degewo AG                                 6

In [310]:
# still not correct identified tables
ben_df3['wrong'].drop(columns=['page_identified']).drop_duplicates().merge(
    ben_df3['correct'].drop(columns=['page_identified']).drop_duplicates(),
    on=['page', 'type', 'company', 'file'],
    how='left',
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge']).merge(
    ben_df3['missing'].drop(columns=['page_identified']).drop_duplicates(),
    on=['page', 'type', 'company', 'file'],
    how='left',
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge'])

,page,type,company,file
0,17,GuV,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
1,71,GuV,IBB,ibb_geschaeftsbericht_2006.pdf
2,24,Aktiva,degewo AG,degewo_Konzernlagebericht_Konzernabschluss_202...
3,25,Passiva,degewo AG,degewo_Konzernlagebericht_Konzernabschluss_202...
4,27,GuV,degewo AG,degewo_Konzernlagebericht_Konzernabschluss_202...
5,28,Aktiva,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
6,29,Passiva,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
7,31,GuV,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
8,43,GuV,Berlin Energie und Netzholding,SIGNATURE_DEE00084236.1.1_550912_Endexemplar_2...


## Exhaustive Regex OCR

In [ ]:
regex_patterns_3 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_3_ocr = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_3,
    verbose = True,
    ocr = True
)

# page_identifier_regex_3_ocr.walk_dir()

In [ ]:
ben_df3_ocr = page_identifier_regex_3_ocr.benchmark()
ben_df3_ocr['missing']

## Exhaustive Regex restricting

In [222]:
# Aktiva und Passiva in einigen AfStatBBB nicht korrekt formatiert
regex_patterns_4 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr",
        r"anlageverm.{1,2}gen"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr",
        r"eigenkapital"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr",
        r"umsatzerl.{1,2}se"
    ]
}

page_identifier_regex_4 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_4,
    verbose = False
)

page_identifier_regex_4.walk_dir()

Mehrseitige GuV in "ibb_geschaeftsbericht_2017.pdf"

In [223]:
ben_df4 = page_identifier_regex_4.benchmark()
ben_df4['missing']

Correctly identified: 184
Wrongly identified: 1121
Missing: 40


,page,type,page_identified,company,file
44,44,GuV,NaN,IBB,ibb_geschaeftsbericht_2017.pdf
47,6,GuV,NaN,IBB,ibb-jahresabschluss-und-lagebericht-2023.pdf
50,57,GuV,NaN,IBB,ibb_geschaeftsbericht_2020.pdf
53,100,GuV,NaN,IBB,ibb_geschaeftsbericht_2013.pdf
56,74,GuV,NaN,IBB,ibb_geschaeftsbericht_2014.pdf
59,71,GuV,NaN,IBB,ibb-geschaeftsbericht-2021.pdf
62,16,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
63,17,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
66,76,GuV,NaN,IBB,ibb_geschaeftsbericht_2007.pdf
69,74,GuV,NaN,IBB,ibb_geschaeftsbericht_2008.pdf


In [224]:
# Aktiva und Passiva in einigen AfStatBBB nicht korrekt formatiert
regex_patterns_5 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr",
        r"anlageverm.{1,2}gen"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr",
        r"eigenkapital"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_5 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_5,
    verbose = False
)

page_identifier_regex_5.walk_dir()

In [225]:
ben_df5 = page_identifier_regex_5.benchmark()
ben_df5['missing']

Correctly identified: 207
Wrongly identified: 1463
Missing: 11


,page,type,page_identified,company,file
187,24,Aktiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
188,25,Passiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
189,27,GuV,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
190,26,Aktiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
191,27,Passiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
192,29,GuV,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
193,26,Aktiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
194,27,Passiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
195,29,GuV,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
196,24,Aktiva,NaN,degewo AG,degewo_Konzernlagebericht_Konzernabschluss_202...


# Tests

## Extracting text

In [301]:
text = PdfReader('../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2006.pdf').pages[67].extract_text()
text

'\x18\x18\nJahresbilanz zum 31. Dezember 2006\na ktivseite in te U r\n31.12.2006 31.12.2005\n1. Barreserve\n\t\n\t\nb)\t\n\t\nGuthaben\n\t\nbei\n\t\nZentralnotenbanken\t\ndarunter:\n\t\nbei\n\t\nder\n\t\nDeutschen\n\t\nBundesbank:\t\nTEUR\n\t\n19.823\n\t\n(31.12.2005\n\t\n:\n\t\nTEUR\n\t\n28.873)\n3. Forderungen an \nk\nreditinstitute\n\t a)\n\t\ntäglich\n\t\nfällig\n\t b)\n\t\nandere\n\t\nForderungen\n4. Forderungen an \nk\nunden\n\t\n\t\ndarunter:\n\t\t\ndurch\n\t\nGrundpfandrechte\n\t\ngesichert:\t\nTEUR\n\t\n9.496.661\n\t\n(31.12.2005\n\t\n:\n\t\nTEUR\n\t\n10.660.277)\t\nKommunalkredite:\n\t\nTEUR\n\t\n3.532.796\n\t\n(31.12.2005\n\t\n:\n\t\nTEUR\n\t\n2.338.961)\n5. Schuldverschreibungen und andere festverzinsliche Wertpapiere\n\t a)\n\t\n\tGeldmarktpapiere\n\t\n\t\t ab)\n\t\nvon\n\t\nanderen\n\t\nEmittenten\t\n\t b)\n\t\n\tAnleihen\n\t\nund\n\t\nSchuldverschreibungen\n\t\n\t\t ba)\n\t\nvon\n\t\nöffentlichen\n\t\nEmittenten\n\t\n\t\t darunter:\n\t\nbeleihbar\n\t\nbei\n\t\nder\n\t\nD

In [305]:
re.search(r"a\sktiva", text, re.IGNORECASE | re.S)

<re.Match object; span=(1380, 1387), match='a\nktiva'>

## OCR

In [231]:
file_list = []
for root, _, files in os.walk("../Geschaeftsberichte/"):
    for file in files:
        file_list.append(os.path.join(root, file))

file_list

missing_files = [file for file in file_list if file not in df_truth['filepath'].values]
missing_files

['../Geschaeftsberichte/degewo AG/240618-degewo-konzernlagerbericht-konzernabschluss-2023.pdf']

The is no table of interest in  "../Geschaeftsberichte/degewo AG/240618-degewo-konzernlagerbericht-konzernabschluss-2023.pdf"

In [235]:
df_truth = pd.read_csv("../Python/table_page_truth.csv")
df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type')
df_truth.shape[0]

265